In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('image', cmap='gray')
import matplotlib.animation

%matplotlib ipympl
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import cv2
from tqdm import tqdm
import random
from stardist import _draw_polygons
from stardist.models import StarDist2D
from csbdeep.utils import normalize
import skimage
from scipy.optimize import dual_annealing, minimize
from tifffile import imsave, imread
from utils import filter_detection_data, get_frame #detect_features,
import trackpy as tp

In [15]:

def detect_features_frame(feature_properties_dict, frame):
    img = get_frame(video, frame, xmin, ymin, xmax, ymax, w, h, True)
    segmented_image, dict_test = model.predict_instances(normalize(img), predict_kwargs = {'verbose' : False})

    feature_properties = skimage.measure.regionprops_table(segmented_image, \
                                                            properties=('area', 'area_bbox', 'area_convex', 'area_filled',\
                                                                        'axis_major_length', 'axis_minor_length',\
                                                                        'bbox', 'centroid', 'eccentricity', \
                                                                        'equivalent_diameter_area', 'euler_number', 'extent',\
                                                                        'feret_diameter_max', 'inertia_tensor',\
                                                                        'inertia_tensor_eigvals', 'label'))

    for key in feature_properties.keys():
        feature_properties_dict[key] += list(feature_properties[key])
    feature_properties_dict['prob']  += list(dict_test['prob'])
    feature_properties_dict['frame'] += list(np.ones(len(list(feature_properties['centroid-0'])))*frame)
    return feature_properties_dict


def detect_features(frames, test_verb):
    feature_properties_dict = {'frame':[], 'centroid-1':[], 'centroid-0':[], 'area':[], 'r':[], 'eccentricity':[],\
                                'prob':[], 'area_bbox':[], 'area_convex':[], 'area_filled':[], 'axis_major_length':[],\
                                'axis_minor_length':[], 'bbox-0':[], 'bbox-1':[], 'bbox-2':[], 'bbox-3':[],\
                                'equivalent_diameter_area':[], 'euler_number':[], 'extent':[], 'feret_diameter_max':[],\
                                'inertia_tensor-0-0':[], 'inertia_tensor-0-1':[], 'inertia_tensor-1-0':[],\
                                'inertia_tensor-1-1':[], 'inertia_tensor_eigvals-0':[], 'inertia_tensor_eigvals-1':[],\
                                'label':[]}

    #test = parallel(detect_features_frame(feature_properties_dict, frame) for frame in tqdm(frames))
    for frame in tqdm(frames):
        feature_properties_dict = detect_features_frame(feature_properties_dict, frame)

    feature_properties_dict['r'] = np.sqrt(np.array(feature_properties_dict['area'])/np.pi)
    raw_detection_df = pd.DataFrame(feature_properties_dict)
    raw_detection_df.rename(columns={'centroid-0': 'y', 'centroid-1': 'x'}, inplace=True)
    raw_detection_df['frame'] = raw_detection_df.frame.astype('int')
    raw_detection_df.sort_values(by=['frame', 'prob'], ascending=[True, False], inplace=True)
    if test_verb:
        raw_detection_df.to_parquet(save_path + f'raw_detection_{video_selection}_{model_name}_test.parquet')
    else:
        raw_detection_df.to_parquet(save_path + f'raw_detection_{video_selection}_{model_name}_{frames[0]}_{frames[-1]}.parquet')
    return raw_detection_df

In [10]:
#model_name = 'stardist_trained'          # stardist model trained for 50 epochs on simulated synthetic dataset
model_name = 'modified_2D_versatile_fluo' # stardist model trained for 150 epochs on simulated dataset starting from the pretrained 2D versatile fluo model
model = StarDist2D(None, name = model_name, basedir = './models/')

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.989263, nms_thresh=0.3.


In [11]:
video_selection = "25b25r-1" # "49b1r"
if video_selection == "25b25r-1":
    xmin, ymin, xmax, ymax = 95, 30, 535, 470    
    save_path = './25b25r/synthetic_simulation/'
elif video_selection == "49b1r":
    xmin, ymin, xmax, ymax = 20, 50, 900, 930

save_path       = f'./{video_selection}/{model_name}/'
source_path     = f'./data/{video_selection}.mp4'
system_name     = f'{video_selection} system'
nDrops = 50

video = cv2.VideoCapture(source_path)
video.set(cv2.CAP_PROP_POS_FRAMES, 0)
w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
n_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video has {n_frames} frames with a resolution of {w}x{h} and a framerate of {fps} fps")

Video has 540000 frames with a resolution of 640x480 and a framerate of 30 fps


In [19]:
def test_detection(n_samples, n_frames, video):
    sample_frames = np.sort(np.random.choice(np.arange(0, n_frames, 1, dtype=int), n_samples, replace=False))
    raw_detection_df = detect_features(frames = sample_frames, test_verb = True)

    n_feature_per_frame = raw_detection_df.groupby('frame').count().x.values
    fig, ax = plt.subplots(2, 2, figsize = (8, 4))
    ax[0, 0].plot(raw_detection_df.frame.unique(), n_feature_per_frame, '.')
    ax[0, 0].set(xlabel = 'Frame', ylabel = 'N of droplets', title = 'N of droplets per frame')
    ax[0, 1].plot(raw_detection_df.r, '.')
    ax[0, 1].set(xlabel = 'Feature index', ylabel = 'Radius [px]', title = 'Radius of features detected')
    ax[1, 0].scatter(raw_detection_df.r, raw_detection_df.eccentricity, s=0.1)
    ax[1, 0].set(xlabel = 'Radius [px]', ylabel='Eccentricity', title='R-eccentricity correlation')
    ax[1, 1].scatter(raw_detection_df.r, raw_detection_df.prob, s=0.1)
    ax[1, 1].set(xlabel = 'Radius [px]', ylabel='Probability', title='R-Probability correlation')
    plt.tight_layout()
    plt.savefig(save_path + f'test.png', dpi = 500)
    plt.close()

    try:
        selected_frame = sample_frames[np.where(raw_detection_df.groupby('frame').count().x.values < nDrops)[0][0]]
        img = get_frame(video, selected_frame, xmin, ymin, xmax, ymax, w, h, True)
        fig, ax = plt.subplots(1, 1, figsize=(5, 5))
        ax.set_title(f"Example of spurious effect at frame {selected_frame}")
        ax.imshow(img, cmap='gray')
        for i in range(len(raw_detection_df.loc[raw_detection_df.frame == selected_frame])):
            ax.add_artist(plt.Circle((raw_detection_df.loc[raw_detection_df.frame == selected_frame].x.values[i], raw_detection_df.loc[raw_detection_df.frame == selected_frame].y.values[i]), \
                                        raw_detection_df.loc[raw_detection_df.frame == selected_frame].r.values[i], color='r', fill=False))
        plt.savefig(save_path + f'example_of_spurious_effect.png', dpi = 500)
        plt.close()
    except:
        print('No spurious effect detected')


In [20]:
test_detection(100, n_frames, video)

100%|██████████| 100/100 [00:19<00:00,  5.25it/s]


No spurious effect detected


In [23]:
if 1:
    startFrame = 10000 #0
    endFrame = 10000 + 10000
    print(f"Processing from {int(startFrame/fps)} s to {int(endFrame/fps)} s")
    sample_frames = np.arange(startFrame, endFrame, 1, dtype=int)
    raw_detection_df = detect_features(frames = sample_frames, test_verb = False)
else:
    raw_detection_df = pd.read_parquet(save_path + "raw_detection_25b-25r_modified_2D_versatile_fluo_0_10000.parquet")
    sample_frames = raw_detection_df.frame.unique()

Processing from 333 s to 666 s


  1%|▏         | 147/10000 [00:27<31:01,  5.29it/s]


KeyboardInterrupt: 

In [ ]:
# analyze the result of raw features location
n_feature_per_frame = raw_detection_df.groupby('frame').count().x.values

fig, ax = plt.subplots(2, 2, figsize = (8, 4))
ax[0, 0].plot(raw_detection_df.frame.unique(), n_feature_per_frame, '.')
ax[0, 0].set(xlabel = 'Frame', ylabel = 'N of droplets', title = 'N of droplets per frame')
ax[0, 1].plot(raw_detection_df.r, '.')
ax[0, 1].set(xlabel = 'Feature index', ylabel = 'Radius [px]', title = 'Radius of features detected')
ax[1, 0].scatter(raw_detection_df.r, raw_detection_df.eccentricity, s=0.1)
ax[1, 0].set(xlabel = 'Radius [px]', ylabel='Eccentricity', title='R-eccentricity correlation')
ax[1, 1].scatter(raw_detection_df.r, raw_detection_df.prob, s=0.1)
ax[1, 1].set(xlabel = 'Radius [px]', ylabel='Probability', title='R-Probability correlation')
plt.tight_layout()
plt.savefig(save_path + f'raw_features.png', dpi = 500)
plt.close()

try:
    selected_frame = sample_frames[np.where(raw_detection_df.groupby('frame').count().x.values < nDrops)[0][0]]
except:
    selected_frame = sample_frames[np.where(raw_detection_df.groupby('frame').count().x.values != nDrops)[0][0]]

img = get_frame(video, selected_frame, xmin, ymin, xmax, ymax, w, h, True)
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.set_title(f"Example of spurious effect at frame {selected_frame}")
ax.imshow(img, cmap='gray')
for i in range(len(raw_detection_df.loc[raw_detection_df.frame == selected_frame])):
    ax.add_artist(plt.Circle((raw_detection_df.loc[raw_detection_df.frame == selected_frame].x.values[i], raw_detection_df.loc[raw_detection_df.frame == selected_frame].y.values[i]), \
                                raw_detection_df.loc[raw_detection_df.frame == selected_frame].r.values[i], color='r', fill=False))
plt.savefig(save_path + f'example_of_spurious_effect.png', dpi = 500)
plt.close()

In [ ]:
if 0:
    cutoff = 100
    mem = 5
    t = tp.link_df(raw_detection_df, cutoff, memory = mem, link_strategy = 'hybrid', neighbor_strategy = 'KDTree', adaptive_stop = 1)
    #print(t)
    t = t.sort_values(['frame', 'particle'])
    trajectories = tp.filter_stubs(t, 25)
    # CREATE COLOR COLUMN AND SAVE DF
    n = max(trajectories.particle)
    print(f"N of droplets: {n + 1}")
    random.seed(5)
    colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(n)]
    for i in range(max(trajectories.particle)+1-n):
        colors.append("#00FFFF")
    c = []
    for p in t.particle:
        c.append(colors[p])
    trajectories["color"] = c
    trajectories.to_parquet(save_path + "raw_tracking_25b-25r_modified_2D_versatile_fluo_0_10000.parquet")
else:
    trajectories = pd.read_parquet(save_path + "raw_tracking_25b-25r_modified_2D_versatile_fluo_0_10000.parquet")

In [ ]:
fig, ax = fig, ax = plt.subplots(1, 1, figsize=(10, 10))
def update_graph(frame):
    df = trajectories.loc[(trajectories.frame == frame), ["x", "y", "color", "r"]]
    for i in range(len(df)):
        graph[i].center = (df.x.values[i], df.y.values[i])
        graph[i].radius = df.r.values[i]
    graph2.set_data(get_frame(video, frame, xmin, ymin, xmax, ymax, w, h, False))
    title.set_text(f'{system_name} Tracking -- t = {round(frame/fps, 1)} s')
    return graph

title = ax.set_title(f'{system_name} Tracking -- t = {0} s')
ax.set(xlabel = 'X [px]', ylabel = 'Y [px]')
df = trajectories.loc[(trajectories.frame == 0), ["x", "y", "color", "r"]]

graph = []
for i in range(len(df)):
    graph.append(ax.add_artist(plt.Circle((df.x.values[i], df.y.values[i]), df.r.values[i], color = df.color.values[i],\
                                           fill = False, linewidth=1)))
graph2 = ax.imshow(get_frame(video, 0, xmin, ymin, xmax, ymax, w, h, False))

ani = matplotlib.animation.FuncAnimation(fig, update_graph, trajectories.frame.unique(), interval = 5, blit=False)
writer = matplotlib.animation.FFMpegWriter(fps = 30, metadata = dict(artist='Matteo Scandola'), extra_args=['-vcodec', 'libx264'])
ani.save(f'./{save_path}/tracking_video.mp4', writer=writer, dpi = 200)
plt.close()